# Most requested app profiles

We are working for a company that builds Android and IOS mobile apps available on Google Play and Apple Store.
They only develop apps that are free to download and install , so our main source of revenue consist of app ads.
The more users that see ads , the better will be our profit.


Our aim in this project is to analyze all english apps on Google Play and Apple Store in order to understand what are the features that make an app desirable for the users , we want to help our developers to understand what type of apps are likely to attract more users. These are two data sets that seems suitable for our goals:


* [A data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data for about 10,000 Android apps from the Google Play. Data was collected in August 2018. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data for about 7,000 iOS apps from the App Store. Data was collected in July 2017. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

In [1]:
opened_file=open('AppleStore.csv')
from csv import reader
read_file=reader(opened_file)
ios=list(read_file)
ios_header=ios[0]
ios=ios[1:]

In [2]:
opened_file=open('googleplaystore.csv')
from csv import reader
read_file=reader(opened_file)
android=list(read_file)
android_header=android[0]
android=android[1:]


To make it easier to explore the data set we'll build a function called explore_data() that we can use to explore rows in a easier way. We'll also add an option to show the number of rows and columns for each data set.

In [3]:
def explore_data(dataset,start,end,rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:',len(dataset))
        print('Number of columns:',len(dataset[0]))
    
   
    

In [4]:
print(ios_header)
print('\n')
(explore_data(ios,0,3,True))

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


In the ios database there are 7197 rows and 16 columns , the columns that seems to be significant for our analysis are : 'track_name', 'currency' , 'price' , rating_count_tot' , 'rating_count_ver' , ' prime_genre'.
Not all column names are clear in this case , details about the content of each column can be found here: [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)

In [5]:
print(android_header)
print('\n')
(explore_data(android,0,3,True))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


In the android database there are 10841 rows and 13 columns , the columns that seems to be interesting for our analysis are : 'Category' , 'App' , 'Type' , 'Installs' , 'Review' , 'Price' , 'Genres'.

# Deleting wrong data

In [6]:
print(len(android))

10841


The row 10472 correspond to the app ' Life made WI-FI Touchscreen Photo Frame' and we can see that its rating (column 3) is 19. This is an absurd because maximum possible rating for Google Play apps is 5. As a consequences we will delete this row.

In [7]:
del(android[10472])
print(len(android))

10840


# Removing Duplicates Entries

## Part One

In the Google Play data set can happen you could find duplicates entries. To optimize our study we don't want to count certain apps more than once so we need to remove the duplicates. For instance Instagram has 4 entries:

In [8]:
for app in android:
    if app[0] == 'Instagram':
        print(app)
        print('\n')

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']




Now we will identify all the duplicates in the Google Play data set.

In [9]:
duplicate_apps=[]
unique_apps=[]

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicates apps:',len(duplicate_apps))
print('\n')


Number of duplicates apps: 1181




However we don't want to remove the duplicates randomly , there is a better strategy. If we examine the Instagram example above , the main differences happens on the fourth position of each row. The different numbers show the data was collected at different times. The higher the number of reviews , the more recent the data should be , so we should maintain the app with the highest number and delete all the others.


## Part Two

Let's start by building the dictionary.

In [10]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
    
    
    

In the previous code cell we found that there are 1181 duplicates apps , so the lenght of our dictionary should be equal to the difference between the lenght of our data set and 1181.
Let's check !


In [11]:
print('Expected length:', len(android)-1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


Now we want to remove the duplicate rows using the reviews_max dictionary. For the duplicate cases we will only keep the entries with the highest number of reviews.


In [12]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)
    

Now let's explore the android_clean data set to confirm that the
number of rows is 9659

In [13]:
print(len(android_clean))



9659


As we expected , android_clean data set has 9659 rows just like reviews_max dictionary

# Removing non-English apps

## Part One

We use English for apps we develop at our company, and we would like to analyze only the apps that are directed towars an English-speaking audience.

We are going to build a function that tell us if an app is English-written or not. To do that we will use the ASCII code knowing that the characters used in an English text are all in the range 0 to 127. If a character exceed this range we'll assume that the app is not an English-written one.

In [14]:
def english_or_not (string):
    for character in string:
        if(ord(character)>127):
            return False
    return True    
        
            
    

In [15]:
print(english_or_not('Instagram'))
print(english_or_not('爱奇艺PPS -《欢乐颂2》电视剧热播'))


True
False


The function seems to work fine , but unfortunately there are many apps that use not common character (™,— (em dash), – (en dash), etc.) or emojis and we would't wish for this to happen.

In [16]:
print(english_or_not('Docs To Go™ Free Office Suite'))
print(english_or_not('Instachat 😜'))

False
False


In [17]:
print(ord('™'))
print(ord('😜'))

8482
128540


## Part Two

To minimize the effects of data loss , we'll remove only apps with a minimum of 4 characters in their names that exceed the considered range

In [18]:
def english_or_not (string):
    count=0 
    
    for character in string:
        if(ord(character)>127):
            count+=1
            
    if (count>3):
        return False
    else:
        return True
    
print(english_or_not('Docs To Go™ Free Office Suite'))
print(english_or_not('Instachat 😜'))  
print(english_or_not('爱奇艺PPS -《欢乐颂2》电视剧热播'))
           
       
        

True
True
False


Now we'll use the english_or_not function to filter out the non-English apps for both data set. 

In [19]:
android_english= []
ios_english= []

for app in android_clean:
    if(english_or_not(app[0])):
        android_english.append(app)

for app in ios:
    if(english_or_not(app[1])):
        ios_english.append(app)
        
print(explore_data(android_english,0,3,True))
print("\n")
print(explore_data(ios_english,0,3,True))
        

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13
None


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '

We can see now we have 9614 Android apps and 6183 iOS apps.  

# Isolating the free apps

As we mentioned in the introduction , we only develop apps that 
are free to download , so we want to isolate the free apps in order to make a more accurate analysis.

In [20]:
android_final = []
ios_final = []

for app in android_english:
    if(app[7] == '0'):
        android_final.append(app)
        
for app in ios_english:
    if(app[4] == '0.0'):
        ios_final.append(app)      
        
print("Android apps:",len(android_final))
print("iOS apps:",len(ios_final))
    

Android apps: 8864
iOS apps: 3222


We are left with 8864 Android apps and 3222 iOS apps , with should be enough for our analysis.

# Most common Apps by genre

## Part One

Our aim is to determinate the kinds of apps that are likely to attract more users because our income is strictly related to the number of people using our apps.

To minimize risks we use this strategy:

  1. Build a minimal Android version of the app and add it to the      Google play.
  2. If the app has a good response from users,we develop it          further.
  3. If the app is profitable after 6 months , we build an iOS        version of the app and add it to the App Store.

Our real goal is to add the app both to the Google Play and the App Store , so we need to find app profiles that are successful on both markets. Let's begin the analisys to understand what are the best app genres to reach our goal. We will need to build frequency tables for *Category* and *Genres* columns of the Google Play data set and for *prime_genre* column of the App Store.
  




## Part Two

Now we want to create two function to analyze the frequency tables:
* One function to build frequency tables that show percentages
* Another function that we can use to display the percentage in a descendent order.
        
        

In [21]:
def freq_table (dataset,index):
    genres= {}
    genres_percentages = {}
    total = 0
    
    for row in dataset:
        total += 1
        value=row[index]
        if value in genres:
            genres[value] += 1
        else:
            genres[value]=1
            
    for key in genres:
        percentage = (genres[key]/total) * 100
        genres_percentages[key] = percentage
        
    return genres_percentages        
        
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])        
    

## Part Three

We start analyzing the frequency table for the *prime_genre* column of the Apple Store data set.

In [22]:
display_table(ios_final,-5)    #prime_genre

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The result shows that in the Apple Store (free English apps) the most common genre is *Games* with the 58.16% of the total.
*Entertainment* are the 7.88% followed by *Photo* and *Education* respectively with the 4.96% and the 3.66%.
The general impression is that the App Store (free English apps) is dominated by fun apps (*Games, Entertainment, Social Networking, Sport, etc.*) while apps with practical purpose (*Health, Utilities, Medical, Book ,etc.*) are more rare.

Lets' continue examinating *Category* and *Genres* columns of the Google Play data set.

In [23]:
display_table(android_final,1)   #Category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

Here the situation is much different. In the Google play seems that there aren't so many apps designed for fun and on the other hand there are a good number of apps designed for practical purpose (family, tools, business, lifestyle) which are therefore better represented on Google Play compared to App Store.
Let's see what the frequency table of the genre column tell us.

In [24]:
display_table(android_final,9)    #Genres

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The difference between the Category and Genres columns is not so clear. However we can surely say that the App Store is dominated by apps designed for fun while Google Play shows a more bilanced profile. Now we would like to get an idea about the kind of apps that have more users. 

# Most popular apps by Genres on the App Store

Now we would like to get an idea about the kind of apps with the most users. For the Google play we can find this information in the *Installs* column , but this information is missing in the App Store data set. As a solution, we'll take the total number of user ratings which we can find in the *rating_count_tot* column. 

In [25]:
genres_ios = freq_table(ios_final,-5)

for genre in genres_ios:
    total=0
    len_genre=0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:
            number_ratings=float(app[5])
            total += number_ratings
            len_genre += 1
    average_n_ratings = total/len_genre
    print(genre , ':' , average_n_ratings)
            

Book : 39758.5
Medical : 612.0
Education : 7003.983050847458
Navigation : 86090.33333333333
Entertainment : 14029.830708661417
Games : 22788.6696905016
Lifestyle : 16485.764705882353
Music : 57326.530303030304
Photo & Video : 28441.54375
Social Networking : 71548.34905660378
Food & Drink : 33333.92307692308
Reference : 74942.11111111111
Finance : 31467.944444444445
Utilities : 18684.456790123455
Health & Fitness : 23298.015384615384
Shopping : 26919.690476190477
News : 21248.023255813954
Weather : 52279.892857142855
Sports : 23008.898550724636
Business : 7491.117647058823
Productivity : 21028.410714285714
Travel : 28243.8
Catalogs : 4004.0


Let's try to analyze the results for the best genres:
* *Navigation, Social Networking , Music and Reference* have the best scores but as we can see below, their results are strongly influenced by few big companies (Facebook, Pinterest, WhatsApp for *Social Networking* , Waze , Google Maps for *Navigation* etc.) , that make these genres seem better for our purpose than they really are.

In [31]:
for app in ios_final:
    if app[-5]=='Social Networking':
        print(app[1], ':', app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

In [32]:
for app in ios_final:
    if app[-5]=='Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


* *Weather* has a very good results but people generally don't spend so much time in these kind of apps , so despite the fact that these kind of apps are popular our revenue from the ads should not be so high as we would expect.
* *Food & Drink* has some potential but to make a successful food & drink app is necessary to have a company in the food field and a delivery service , in fact some big examples here are McDonald's , Starbucks etc.
* *Finance* should be another good choice but building a finance app requires domain knowledge and we wouldn't want to hire a finance expert. 

Let's see deeply the *Health & Fitness* and *Sports* genres:


In [34]:
for app in ios_final:
    if app[-5]=='Health & Fitness':
        print(app[1], ':', app[5])

Calorie Counter & Diet Tracker by MyFitnessPal : 507706
Lose It! – Weight Loss Program and Calorie Counter : 373835
Weight Watchers : 136833
Sleep Cycle alarm clock : 104539
Fitbit : 90496
Period Tracker Lite : 53620
Nike+ Training Club - Workouts & Fitness Plans : 33969
Plant Nanny - Water Reminder with Cute Plants : 27421
Sworkit - Custom Workouts for Exercise & Fitness : 16819
Clue Period Tracker: Period & Ovulation Tracker : 13436
Headspace : 12819
Fooducate - Lose Weight, Eat Healthy,Get Motivated : 11875
Runtastic Running, Jogging and Walking Tracker : 10298
WebMD for iPad : 9142
8fit - Workouts, meal plans and personal trainer : 8730
Garmin Connect™ Mobile : 8341
Record by Under Armour, connects with UA HealthBox : 7754
Fitstar Personal Trainer : 7496
My Cycles Period and Ovulation Tracker : 7469
Seven - 7 Minute Workout Training Challenge : 6808
RUNNING for weight loss: workout & meal plans : 6407
Lifesum – Inspiring healthy lifestyle app : 5795
Waterlogged - Daily Hydration Tr

In [37]:
for app in ios_final:
    if app[-5]=='Sports':
        print(app[1], ':', app[5])

ESPN: Get scores, news, alerts & watch live sports : 290996
Yahoo Fantasy Sports : 190670
WatchESPN : 159735
The Masters Tournament : 148160
Yahoo Sports - Teams, Scores, News & Highlights : 137951
ESPN Fantasy Football Baseball Basketball Hockey : 64925
CBS Sports App - Sports Scores, News, Stats, Watch : 59639
FOX Sports Mobile : 57500
2016 U.S. Open Golf Championship : 54192
NBC Sports : 47172
NBA : 43682
ESPN Tournament Challenge : 39642
2016 US Open Tennis Championships : 37522
NFL : 27317
MLB.com At Bat : 21830
The Championships, Wimbledon 2016 - Tennis Grand Slam : 20953
DraftKings - Daily Fantasy Golf, Baseball, & More : 20251
Bleacher Report: Sports news, scores, & highlights : 16979
Univision Deportes: Liga MX, MLS, Fútbol En Vivo : 16683
NASCAR MOBILE : 16385
NHL : 15554
FOX Sports GO : 13898
NCAA March Madness Live - Men's College Basketball : 13572
PGA Championship 2016 – Baltusrol Golf Club : 10472
365Scores : 9879
NFL Fantasy Football - Official NFL Fantasy App : 8234
Fa

*Sports* and *Health & Fitness* don't have the best results in general , but as we can see there a lot of apps with a quite good score. For example in *Sports* there are 17 apps with a score greater or equal to 20,000.  

We note that *Games* are absolutely the genre with the most number of apps in the App Store (free-English apps) but the market might be a bit saturated as we can see from the last results : *Games* has a score less than 23,000.

Now let's analyze the Google Play market as well.

# Most popular apps by Genre on the Google Play

For the Google Play market we can get information about the number of installs so we should be able to obtain a clearer picture of the situation. 

In [39]:
display_table(android_final,5)    # the Installs column

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


The problem with this data is that it's not precise. Anyway we only want to get an idea of which app genres attract the most of users so we , we don't need perfect precision.
We'll consider that an app with 100,000+ installs has 100,000 installs , an app with 1,000,000+ installs has 1,000,000 installs and so on.
To perform computation we need to convert each install number from string to float removing commas and the plus characthers.
We'll do this directly in the loop below and we also compute the average number of installs for each genre.

In [40]:
category_android = freq_table(android_final,1)

for category in category_android:
    total=0
    len_category=0
    for app in android_final:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+','')
            n_installs = n_installs.replace(',','')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category,':',avg_n_installs)
            



BEAUTY : 513151.88679245283
COMICS : 817657.2727272727
SHOPPING : 7036877.311557789
ENTERTAINMENT : 11640705.88235294
AUTO_AND_VEHICLES : 647317.8170731707
EVENTS : 253542.22222222222
PRODUCTIVITY : 16787331.344927534
FAMILY : 3695641.8198090694
HEALTH_AND_FITNESS : 4188821.9853479853
COMMUNICATION : 38456119.167247385
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
FINANCE : 1387692.475609756
PERSONALIZATION : 5201482.6122448975
MEDICAL : 120550.61980830671
PARENTING : 542603.6206896552
SOCIAL : 23253652.127118643
FOOD_AND_DRINK : 1924897.7363636363
BOOKS_AND_REFERENCE : 8767811.894736841
LIBRARIES_AND_DEMO : 638503.734939759
VIDEO_PLAYERS : 24727872.452830188
ART_AND_DESIGN : 1986335.0877192982
NEWS_AND_MAGAZINES : 9549178.467741935
PHOTOGRAPHY : 17840110.40229885
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
MAPS_AND_NAVIGATION : 4056941.7741935486
SPORTS : 3638640.1428571427
WEATHER : 507

On average , the categories with best results are *Communication* and *Video_Players*. 

In [41]:
for app in android_final:
    if (app[1] == 'COMMUNICATION') and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

We can make the same consideration made previously , there are some few big companies that strongly influence the market.
Now let's have a look to smaller categories , for example the sports one.

In [44]:
for app in android_final:
    if app[1] == 'SPORTS': 
        print(app[0], ':', app[5])

Free Sports TV : 100,000+
LiveScore: Live Sport Updates : 10,000,000+
Onefootball - Soccer Scores : 10,000,000+
Cristiano Ronaldo Wallpaper : 500,000+
Futbol24 : 1,000,000+
kicker football news : 5,000,000+
Football Live Scores : 5,000,000+
Pro 2018 - Series A and B : 5,000,000+
BeSoccer - Soccer Live Score : 10,000,000+
Sport.pl LIVE : 1,000,000+
The Team - Live Sport: football, tennis, rugby .. : 5,000,000+
MARCA - Sports Leader Diary : 5,000,000+
Sahadan Live Scores : 5,000,000+
Foot Mercato: transfers, results, news, live : 1,000,000+
All Football - Latest News & Videos : 10,000,000+
Eurosport : 10,000,000+
beIN SPORTS TR : 10,000,000+
All Football GO- Live Score, Games : 100,000+
Top Mercato: football news : 1,000,000+
GollerCepte Live Score : 1,000,000+
Championat : 1,000,000+
Hovercraft Racer : 100,000+
World Cup 2018 : 500,000+
GollerCepte 1903 : 500,000+
League18 : 500,000+
Speed Boat Racing : 1,000,000+
La Liga - Spanish Soccer League Official : 10,000,000+
Lionel Messi Wallp

In [42]:
for app in android_final:
    if app[1] == 'SPORTS' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

3D Bowling : 100,000,000+
FIFA Soccer : 100,000,000+


Here the situation is much different. There aren't big companies that dominates the market as we can see from the fact that there are only 2 apps that have 100,000,000+ installs.

# Conclusion

In this project we analyzed data from the App Store and the Google Play with the goal of recommanding an app profile that can be profitable for both markets.


We concluded that an app which is about sport could be a good choice.The market of this kind of apps is not dominated by big companies, and so there is some potential. Best sports choice seems to be football and basketball beacuse of their big popularity. 